In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
spark = SparkSession.builder.appName("SparkHomework").getOrCreate()

spark

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")
spark.conf.set("spark.sql.bucketing.enabled", "1")

maps = spark.read.option("header", "true").csv("/home/iceberg/data/maps.csv")
medals = spark.read.option("header","true").csv("/home/iceberg/data/medals.csv")


25/08/27 19:20:54 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
print(maps.count())
# maps.show()

40


In [3]:
print(medals.count())
# medals.show()

183


In [4]:
from pyspark.sql.functions import broadcast

medals_with_maps = medals.join(broadcast(maps),"name")
medals_with_maps.show()

+----+--------+----------+-----------+----------+------------------+-------------------+------------+-------------+--------------+-----------+----------+-----+-----------+
|name|medal_id|sprite_uri|sprite_left|sprite_top|sprite_sheet_width|sprite_sheet_height|sprite_width|sprite_height|classification|description|difficulty|mapid|description|
+----+--------+----------+-----------+----------+------------------+-------------------+------------+-------------+--------------+-----------+----------+-----+-----------+
+----+--------+----------+-----------+----------+------------------+-------------------+------------+-------------+--------------+-----------+----------+-----+-----------+



In [5]:
from pyspark.sql.functions import bucket


matches = spark.read.option("header", "true").option("inferSchema", "true").csv("/home/iceberg/data/matches.csv")
# matches.show()


In [6]:
%%sql
DROP TABLE IF EXISTS bootcamp.matches_bucketed

25/08/27 19:21:03 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [7]:
%%sql
CREATE TABLE
  IF NOT EXISTS bootcamp.matches_bucketed (
    match_id STRING,
    mapid STRING,
    is_team_game BOOLEAN,
    playlist_id STRING,
    game_variant_id STRING,
    is_match_over BOOLEAN,
    completion_date TIMESTAMP,
    match_duration STRING,
    game_mode STRING,
    map_variant_id STRING
  ) USING iceberg PARTITIONED BY (bucket(16, match_id))

++
||
++
++

In [8]:
matches.write.mode("append").bucketBy(16,"match_id").saveAsTable("bootcamp.matches_bucketed")

In [9]:
%%sql
SELECT
  COUNT(1)
FROM
  bootcamp.matches_bucketed.files

count(1)
16


In [10]:

match_details = spark.read.option("header","true").option("inferSchema", "true").csv("/home/iceberg/data/match_details.csv")
# match_details.show()


In [11]:
%%sql
DROP TABLE IF EXISTS bootcamp.match_details_bucketed

++
||
++
++

In [12]:
%%sql
CREATE TABLE
  IF NOT EXISTS bootcamp.match_details_bucketed (
    match_id STRING,
    player_gamertag STRING,
    previous_spartan_rank INTEGER,
    spartan_rank INTEGER,
    previous_total_xp INTEGER,
    total_xp INTEGER,
    previous_csr_tier STRING,
    previous_csr_designation STRING,
    previous_csr INTEGER,
    previous_csr_percent_to_next_tier STRING,
    previous_csr_rank INTEGER,
    current_csr_tier STRING,
    current_csr_designation STRING,
    current_csr INTEGER,
    current_csr_percent_to_next_tier STRING,
    current_csr_rank INTEGER,
    player_rank_on_team STRING,
    player_finished BOOLEAN,
    player_average_life STRING,
    player_total_kills INTEGER,
    player_total_headshots INTEGER,
    player_total_weapon_damage FLOAT,
    player_total_shots_landed INTEGER,
    player_total_melee_kills INTEGER,
    player_total_melee_damage FLOAT,
    player_total_assassinations INTEGER,
    player_total_ground_pound_kills INTEGER,
    player_total_shoulder_bash_kills INTEGER,
    player_total_grenade_damage FLOAT,
    player_total_power_weapon_damage FLOAT,
    player_total_power_weapon_grabs INTEGER,
    player_total_deaths INTEGER,
    player_total_assists INTEGER,
    player_total_grenade_kills INTEGER,
    did_win INTEGER,
    team_id STRING
  ) USING iceberg PARTITIONED BY (bucket(16, match_id));

25/08/27 19:21:12 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


++
||
++
++

In [13]:
match_details.write.mode("append").bucketBy(16,"match_id").saveAsTable("bootcamp.match_details_bucketed")

In [14]:
%%sql
SELECT
  SUM(file_size_in_bytes) AS size,
  COUNT(1) AS num_files,
  'sorted'
FROM
  bootcamp.match_details_bucketed.files

size,num_files,sorted
7136552,16,sorted


In [15]:

medals_matches_players = spark.read.option("header","true").option("inferSchema", "true").csv("/home/iceberg/data/medals_matches_players.csv")
# medals_matches_players.show()

In [16]:
%%sql
DROP TABLE IF EXISTS bootcamp.medals_matches_players_bucketed

++
||
++
++

In [17]:
%%sql
CREATE TABLE
  IF NOT EXISTS bootcamp.medals_matches_players_bucketed (
    match_id STRING,
    player_gamertag STRING,
    medal_id STRING,
    COUNT INTEGER
  ) USING iceberg PARTITIONED BY (bucket(16, match_id))

++
||
++
++

In [18]:
medals_matches_players.write.mode("append").bucketBy(16,"match_id").saveAsTable("bootcamp.medals_matches_players_bucketed")

In [19]:
%%sql
SELECT
  SUM(file_size_in_bytes) AS size,
  COUNT(1) AS num_files,
  'sorted'
FROM
  bootcamp.medals_matches_players_bucketed.files

size,num_files,sorted
2912283,16,sorted


In [20]:
match_details_bucketed = spark.table("bootcamp.match_details_bucketed")
matches_bucketed = spark.table("bootcamp.matches_bucketed")
medals_matches_players_bucketed = spark.table("bootcamp.medals_matches_players_bucketed")

df = match_details_bucketed.join(matches_bucketed, "match_id").join(medals_matches_players_bucketed,"match_id")
df.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [match_id#604, player_gamertag#605, previous_spartan_rank#606, spartan_rank#607, previous_total_xp#608, total_xp#609, previous_csr_tier#610, previous_csr_designation#611, previous_csr#612, previous_csr_percent_to_next_tier#613, previous_csr_rank#614, current_csr_tier#615, current_csr_designation#616, current_csr#617, current_csr_percent_to_next_tier#618, current_csr_rank#619, player_rank_on_team#620, player_finished#621, player_average_life#622, player_total_kills#623, player_total_headshots#624, player_total_weapon_damage#625, player_total_shots_landed#626, player_total_melee_kills#627, ... 24 more fields]
   +- SortMergeJoin [match_id#604], [match_id#696], Inner
      :- Project [match_id#604, player_gamertag#605, previous_spartan_rank#606, spartan_rank#607, previous_total_xp#608, total_xp#609, previous_csr_tier#610, previous_csr_designation#611, previous_csr#612, previous_csr_percent_to_next_tier#613, previous_csr_ra